
# Prerequisite

In [ ]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Code, Dataset

In [ ]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

subscription_id = '4aaa645c-5ae2-4ae9-a17a-84b9023bc56a'
resource_group = 'itp-pilot-ResGrp'
workspace = 'itp-pilot-eastus'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

# Basic pipeline job

## Build pipeline

In [ ]:
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from pathlib import Path

parent_dir = ''


def generate_dsl_pipeline() -> Pipeline:
    # 1. Load component funcs
    tf_func = dsl.load_component(yaml_file=parent_dir + "./component.yml")

    # 2. Construct pipeline
    @dsl.pipeline(
        description="Train using TF component",
    )
    def sample_pipeline():
        tf_job = tf_func(epochs=1)
        tf_job.compute = "cpu-cluster"
        tf_job.outputs.trained_model_output.mode = "upload"
        tf_job.distribution.worker_count = 1
        tf_job.distribution.distributionType = "tensorflow"
        tf_job.resources.instance_count = 2

    pipeline = sample_pipeline()
    return pipeline


# Submit pipeline job

In [ ]:
# create pipeline instance
pipeline = generate_dsl_pipeline()
# submit job to workspace
ml_client.jobs.create_or_update(pipeline, experiment_name="tf_mnist", continue_run_on_step_failure=True)